In [21]:
import pandas as pd
import numpy as np
import pickle

In [22]:
# Read data from file
hhold = pd.read_csv('../train/A_hhold_train.csv')
indiv = pd.read_csv('../train/A_indiv_train.csv')

In [23]:
hhold.head()

,id,wBXbHZmp,SlDKnCuu,KAJOWiiw,DsKacCdL,rtPrBBPl,tMJrvvut,jdetlNNF,maLAYXwi,vwpsXRGk,...,sDGibZrP,CsGvKKBJ,OLpGAaEu,LrDrWRjC,JCDeZBXq,HGPWuGlV,GDUPaBQs,WuwrCsIY,AlDbXTlZ,country
0,46107,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,XAmOF,...,JqHnW,MaXfS,etZsD,idRwx,LPtkN,vkbkA,qQxrL,AITFl,aQeIm,A
1,82739,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,lwCkE,...,JqHnW,MaXfS,HxnJy,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
2,9646,JhtDR,GUusz,BIZns,ZYabk,uxuSS,PHMVg,NDTCU,sehIp,qNABl,...,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
3,10975,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,sPNOc,...,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
4,16463,JhtDR,alLXR,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,NdlDR,...,JqHnW,MaXfS,etZsD,idRwx,UyAms,vkbkA,qQxrL,GAZGl,aQeIm,A


In [24]:
indiv.head()

,id,iid,HeUgMnzF,CaukPfUC,MzEtIdUF,gtnNTNam,SWoXNmPc,eXbOkwhI,OdXpbPGJ,XONDGWjH,...,ukWqmeSS,qqVibbSA,MgCoFhXK,rFpoTXAq,RXcLsVAQ,rQWIpTiG,XizJGmbu,xqUooaNJ,poor,country
0,80389,1,XJsPz,mOlYV,UFoKR,SSvEP,onRNG,YXCNt,4.0,oArAw,...,181,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,dSJoN,True,A
1,80389,2,XJsPz,mOlYV,axSTs,CXizI,onRNG,YXCNt,4.0,ccbZA,...,141,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,JTCKs,True,A
2,80389,3,TRFeI,mOlYV,axSTs,CXizI,NDnCs,YXCNt,4.0,fOUHD,...,41,QQdHS,gCSRj,Hikoa,zQvdC,rkLqZ,juMSt,JTCKs,True,A
3,80389,4,XJsPz,yAyAe,FRcdT,CXizI,onRNG,YXCNt,NaN,fOUHD,...,16,QQdHS,uEstx,Hikoa,zQvdC,jVHyH,GtHel,JTCKs,True,A
4,39883,1,XJsPz,mOlYV,UFoKR,HIvIU,onRNG,YXCNt,4.0,oArAw,...,381,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,UaIsy,False,A


In [25]:
def col_mapper (df, **kwargs):
    '''Creates dictionary of original column names (key) and their
    altered names (value). User must pass a Pandas dataframe.
    
    @ Optional Keyword Arguments:
    -----------------------------
    prefix: a string prefix that will be added to all column names
    protected: a list of column names that will not be altered
    zfill_amt: for enumeration, amount of digits (e.g. 3 will result in 001, 002, etc.)
    '''
    # Gather optional keyword arguments
    prefix = kwargs.pop('prefix', '')
    protected = kwargs.pop('protected', [])
    zfill_amt = kwargs.pop('zfill_amt', 3)
    # Initialize counters and temporary dictionary
    num_counter = 1
    cat_counter = 1
    temp = {}
    # Iterate through columns
    for col in df.columns:
        # For protected columns, add to dictionary as is
        if col in protected:
            temp[col] = col
        else:
            # For numeric, rename like "<prefix>num_005"
            if df[col].dtype in ['int64', 'float64']:
                temp[col] = prefix + 'num' + '_' + str(num_counter).zfill(zfill_amt)
                num_counter += 1
            # For categorical, rename like "<prefix>cat_006"
            elif df[col].dtype == 'object':
                temp[col] = prefix + 'cat' + '_' + str(cat_counter).zfill(zfill_amt)
                cat_counter += 1
    return temp

In [26]:
# Create a dictionary of original --> new column names
hhold_column_dict = col_mapper(hhold, prefix='h_', protected=['id', 'iid', 'poor', 'country'])
# Also create a reversed dictionary for easy look-up in other direction
hhold_rev_column_dict = {val: key for key, val in hhold_column_dict.items()}

# Same for indiv
indiv_column_dict = col_mapper(indiv, prefix='i_', protected=['id', 'iid', 'poor', 'country'])
# Also create a reversed dictionary for easy look-up in other direction
indiv_rev_column_dict = {val: key for key, val in indiv_column_dict.items()}

In [27]:
# Make a set of new columns
hhold_new_columns = hhold.columns.map(lambda x: hhold_column_dict[x])
indiv_new_columns = indiv.columns.map(lambda x: indiv_column_dict[x])

In [28]:
# Replace old columns with new
hhold.columns = hhold_new_columns
indiv.columns = indiv_new_columns

In [29]:
# Drop country as variable
hhold.drop(labels=['country'], axis=1, inplace=True)
indiv.drop(labels=['country'], axis=1, inplace=True)

In [30]:
hhold.head()

,id,h_cat_001,h_cat_002,h_cat_003,h_cat_004,h_cat_005,h_cat_006,h_cat_007,h_cat_008,h_cat_009,...,h_cat_330,h_cat_331,h_cat_332,h_cat_333,h_cat_334,h_cat_335,h_cat_336,h_cat_337,h_cat_338,h_cat_339
0,46107,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,XAmOF,...,wxJis,JqHnW,MaXfS,etZsD,idRwx,LPtkN,vkbkA,qQxrL,AITFl,aQeIm
1,82739,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,lwCkE,...,AZwXA,JqHnW,MaXfS,HxnJy,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
2,9646,JhtDR,GUusz,BIZns,ZYabk,uxuSS,PHMVg,NDTCU,sehIp,qNABl,...,AZwXA,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
3,10975,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,sPNOc,...,wxJis,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
4,16463,JhtDR,alLXR,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,NdlDR,...,wxJis,JqHnW,MaXfS,etZsD,idRwx,UyAms,vkbkA,qQxrL,GAZGl,aQeIm


In [31]:
indiv.head()

,id,iid,i_cat_001,i_cat_002,i_cat_003,i_cat_004,i_cat_005,i_cat_006,i_num_001,i_cat_007,...,i_cat_031,i_num_002,i_cat_032,i_cat_033,i_cat_034,i_cat_035,i_cat_036,i_cat_037,i_cat_038,poor
0,80389,1,XJsPz,mOlYV,UFoKR,SSvEP,onRNG,YXCNt,4.0,oArAw,...,yOwsR,181,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,dSJoN,True
1,80389,2,XJsPz,mOlYV,axSTs,CXizI,onRNG,YXCNt,4.0,ccbZA,...,yOwsR,141,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,JTCKs,True
2,80389,3,TRFeI,mOlYV,axSTs,CXizI,NDnCs,YXCNt,4.0,fOUHD,...,yOwsR,41,QQdHS,gCSRj,Hikoa,zQvdC,rkLqZ,juMSt,JTCKs,True
3,80389,4,XJsPz,yAyAe,FRcdT,CXizI,onRNG,YXCNt,NaN,fOUHD,...,yOwsR,16,QQdHS,uEstx,Hikoa,zQvdC,jVHyH,GtHel,JTCKs,True
4,39883,1,XJsPz,mOlYV,UFoKR,HIvIU,onRNG,YXCNt,4.0,oArAw,...,yOwsR,381,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,UaIsy,False


In [32]:
# Get various lists of features for easy pandas slicing
# Indices and target variable are in protected class
protected_cols = ['id', 'iid', 'poor']
# Everything not protected
all_feat_list = [col for col in indiv.columns if col not in protected_cols]
# Numeric
num_feat_list = [col for col in indiv.columns if indiv[col].dtype in ['int64', 'float64'] and col not in protected_cols]
# Categorical
cat_feat_list = [col for col in indiv.columns if indiv[col].dtype == 'object' and col not in protected_cols]

In [33]:
# Return a new dataframe with all categorical features converted into dummies
indiv_dummies = pd.get_dummies(indiv[cat_feat_list])

In [34]:
indiv_dummies.head()

,i_cat_001_BNCcM,i_cat_001_HUpWg,i_cat_001_JMXQx,i_cat_001_PAVsH,i_cat_001_SJPkb,i_cat_001_SlRmt,i_cat_001_TRFeI,i_cat_001_XJgvq,i_cat_001_XJsPz,i_cat_001_YsSBt,...,i_cat_036_rkLqZ,i_cat_036_xUYIC,i_cat_037_FUUXv,i_cat_037_GtHel,i_cat_037_juMSt,i_cat_038_ALcKg,i_cat_038_JTCKs,i_cat_038_UaIsy,i_cat_038_dSJoN,i_cat_038_vhhVz
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0


In [35]:
# Merge the protected and numeric columns with the dummy categorical data
# We keep this named separately from indiv to avoid overwriting original data
indiv = pd.merge(indiv[protected_cols + num_feat_list], indiv_dummies, left_index=True, right_index=True)

In [36]:
indiv.head()

,id,iid,poor,i_num_001,i_num_002,i_cat_001_BNCcM,i_cat_001_HUpWg,i_cat_001_JMXQx,i_cat_001_PAVsH,i_cat_001_SJPkb,...,i_cat_036_rkLqZ,i_cat_036_xUYIC,i_cat_037_FUUXv,i_cat_037_GtHel,i_cat_037_juMSt,i_cat_038_ALcKg,i_cat_038_JTCKs,i_cat_038_UaIsy,i_cat_038_dSJoN,i_cat_038_vhhVz
0,80389,1,True,4.0,181,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1,80389,2,True,4.0,141,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
2,80389,3,True,4.0,41,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
3,80389,4,True,NaN,16,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,39883,1,False,4.0,381,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [38]:
# Test for missing values in numeric features - here we see about 6,000
indiv['i_num_001'].isnull().sum()

6268

In [39]:
# The second numeric feature is clean, though
indiv['i_num_002'].isnull().sum()

0

In [41]:
# There is an extremely high bias for 4.0 in 'num_001', so we will use mode 
# rather than the mean to replace NaNs
indiv['i_num_001'].value_counts()

4.0      29436
100.0      314
84.0       273
88.0        92
64.0        74
116.0       69
20.0        62
124.0       57
172.0       54
24.0        51
74.0        49
104.0       45
76.0        41
28.0        39
94.0        36
148.0       35
112.0       35
16.0        32
44.0        32
52.0        31
144.0       31
18.0        27
102.0       25
54.0        20
40.0        19
14.0        19
6.0         16
60.0        16
22.0        15
96.0        14
         ...  
90.0         4
108.0        4
46.0         4
120.0        4
114.0        3
128.0        3
110.0        3
184.0        3
86.0         2
56.0         2
140.0        2
134.0        2
156.0        2
186.0        2
38.0         2
122.0        2
214.0        1
106.0        1
42.0         1
5.0          1
48.0         1
180.0        1
152.0        1
200.0        1
66.0         1
196.0        1
30.0         1
50.0         1
118.0        1
178.0        1
Name: i_num_001, Length: 82, dtype: int64

In [42]:
# Fill missing values with the mode of 'num_001'
indiv['i_num_001'] = indiv['i_num_001'].fillna(value=indiv['i_num_001'].mode().item())

In [44]:
# Output individual and household data as pickle files
with open('hhold.pickle', 'wb') as f:
    pickle.dump(hhold, f, pickle.HIGHEST_PROTOCOL)
with open('indiv.pickle', 'wb') as f:
    pickle.dump(indiv, f, pickle.HIGHEST_PROTOCOL)